In [2]:
from nltk.grammar import DependencyGrammar
from nltk.parse.dependencygraph import DependencyGraph

In [28]:
import os, random
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import TextVectorization


class DataInput():
    def __init__(self, data_url, train_size, dev_size, sequence_length=200):
        docs= self.import_data(data_url)
        self.sequence_length = sequence_length
        self.datasets = self.train_dev_test_split(docs, train_size, dev_size)
        self.X_vocabulary, self.y_vocabulary = self.setup_vectorizers(*self.datasets[0])
        for i, (X,y) in enumerate(self.datasets):
            self.datasets[i] = self.vectorize_dataset(X,y)
        
    def import_data(self, data_url):
        """
        Import POS dataset from URL.
        """
        !wget -O data.zip $data_url
        !unzip -o data.zip -d data
        !rm data.zip
        self.data_dir = "data/" + os.listdir('data')[0] + "/"
        docs = os.listdir(self.data_dir)
        return docs

    def parse_dataset(self, docs):
        """
        Parse the dependency treebank dataset.
        """
        X = []
        y = []
        for doc in docs:
            np_doc = np.loadtxt(self.data_dir+doc, str, delimiter='\t')
            X.append(" ".join(np_doc[:,0]))
            y.append(" ".join(np_doc[:,1]))
        return np.array(X),np.array(y)

    def train_dev_test_split(self, docs, train_size, dev_size):
        """
        Split dataset into train and test.

        Args:
            docs: list of documents
            train_size: float, percentage of train data
            dev_size: float, percentage of dev data (note that test size is 1-train_size-dev_size)

        Returns:
            train_docs: list of train documents
        """
        random.shuffle(docs)
        print(int(train_size*len(docs)))
        train_docs = self.parse_dataset(docs[:int(train_size*len(docs))])
        dev_docs = self.parse_dataset(docs[int(train_size*len(docs)):int((train_size+dev_size)*len(docs))])
        test_docs = self.parse_dataset(docs[int((train_size+dev_size)*len(docs)):])
        return [train_docs, dev_docs, test_docs]

    def setup_vectorizers(self, X_train, y_train):
        self.X_vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=self.sequence_length)
        self.y_vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=self.sequence_length)
        self.X_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(X_train))
        self.y_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(y_train))
        self.one_hot_depth = len(self.y_vectorizer.get_vocabulary())+1
        return self.X_vectorizer.get_vocabulary(), self.y_vectorizer.get_vocabulary()
    def vectorize_dataset(self, X, y):
        X = self.X_vectorizer(np.array([[s] for s in X])).numpy()
        y = self.y_vectorizer(np.array([[s] for s in y])).numpy()
        y = tf.one_hot(y, self.one_hot_depth)
        return X,y

In [22]:
import glob 

class Embedder():
    def __init__(self, glove_url="http://nlp.stanford.edu/data/glove.6B.zip", embedding_dim=100):
        self.download_if_needed(glove_url)
        self.embedding_dim = embedding_dim
        self.embedding = self.parse_embeddings()
    def download_if_needed(self, glove_url):
        if not glob.glob("glove*.txt"):
            !wget $glove_url
            !unzip -q glove.6B.zip
    def parse_embeddings(self):
        import numpy as np
        embeddings_index = {}
        with open("glove.6B."+str(self.embedding_dim) + "d.txt") as f:
            for line in f:
                word, coefs = line.split(maxsplit=1)
                coefs = np.fromstring(coefs, "f", sep=" ")
                embeddings_index[word] = coefs
        return embeddings_index
    def embedding_matrix(self, vocabulary):
        word_index = dict(zip(vocabulary, range(len(vocabulary))))
        embedding_matrix = np.zeros((len(vocabulary)+2, self.embedding_dim))
        for word, i in word_index.items():
            embedding_vector = self.embedding.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        return embedding_matrix
    
        

In [29]:
di = DataInput("https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip", 0.5, 0.25)


--2022-03-11 11:37:47--  https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457429 (447K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 446.71K  --.-KB/s    in 0.04s   

2022-03-11 11:37:47 (9.80 MB/s) - ‘data.zip’ saved [457429/457429]

Archive:  data.zip
  inflating: data/dependency_treebank/wsj_0093.dp  
  inflating: data/dependency_treebank/wsj_0065.dp  
  inflating: data/dependency_treebank/wsj_0039.dp  
  inflating: data/dependency_treebank/wsj_0182.dp  
  inflating: data/dependency_treebank/wsj_0186.dp  
  inflating: data/dependency_treebank/wsj_0041.dp  
  inflating: data/dependency_treebank/wsj_0018.dp  
  inflati

In [30]:
embedder = Embedder()
embedding_matrix = embedder.embedding_matrix(di.X_vocabulary)
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ...,
       [-0.058569  , -0.50269997,  0.078622  , ...,  0.098051  ,
        -0.61306   , -0.52842999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam

class NeuralNetwork:
    def __init__(self, X_vocabulary, embedding_dim, one_hot_depth, sequence_length, embedding_matrix):
        embedding_layer = Embedding(
            len(X_vocabulary)+2, # Number of tokens in the vocabulary
            embedding_dim, # Dimensions of the embedding,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        )
        self.model = models.Sequential()
        self.model.add(layers.InputLayer(input_shape=(sequence_length,)))
        self.model.add(embedding_layer)
        self.model.add(layers.Bidirectional(layers.LSTM(512, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001))))
        self.model.add(layers.TimeDistributed(layers.Dense(one_hot_depth)))
        self.model.add(layers.Activation('softmax'))
        self.model.compile(loss="binary_crossentropy", optimizer=Adam(0.001),  metrics=["accuracy"])
        self.model.summary()


In [ ]:
nn = NeuralNetwork(di.X_vocabulary, embedder.embedding_dim, di.one_hot_depth, di.sequence_length, embedding_matrix)
X_train, y_train = di.datasets[0]
nn.model.fit(X_train, y_train, epochs=100, batch_size=12, validation_data=di.datasets[1])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 100)          739500    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200, 1024)        2510848   
 nal)                                                            
                                                                 
 time_distributed_2 (TimeDis  (None, 200, 39)          39975     
 tributed)                                                       
                                                                 
 activation_2 (Activation)   (None, 200, 39)           0         
                                                                 
Total params: 3,290,323
Trainable params: 3,290,323
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
9/9 [===========================